In [1]:
import pandas as pd
import numpy as np

In [76]:
lab_sofa=pd.read_csv("Labs_withSOFA.csv")
vent_sofa=pd.read_csv("drugate_norm_updated.csv",usecols=['patientunitstayid','offset','SOFA_cardio'])
gcs_sofa=pd.read_csv("gcs_withSOFA.csv",usecols=['patientunitstayid','offset','SOFA_GCS'])
t_sus=pd.read_csv("ABX_BC_pid_offset_Clean_TsuspicionMax.csv")

In [4]:
lab_sofa.head()

,patientunitstayid,offset,SOFA_Coagulation,SOFA_Liver,SOFA_Respiration,SOFA_Renal
0,141168,516.0,0.0,2.0,0.0,0.0
1,141168,1133.0,0.0,2.0,0.0,2.0
2,141168,1701.0,0.0,0.0,0.0,4.0
3,141168,1805.0,0.0,0.0,2.0,0.0
4,141168,2010.0,0.0,0.0,2.0,0.0


In [5]:
vent_sofa.head()

,patientunitstayid,SOFA_cardio,offset
0,242082,4.0,425
1,242082,3.0,125
2,242082,3.0,55
3,242082,3.0,665
4,242082,3.0,230


In [6]:
gcs_sofa.head()

,patientunitstayid,offset,SOFA_GCS
0,141924,2109,0.0
1,141924,1894,0.0
2,141924,424,0.0
3,141944,484,0.0
4,141944,2389,1.0


In [7]:
t_sus.head()

,patientunitstayid,max
0,242040,188.0
1,242154,8589.0
2,242290,165.0
3,242319,1221.0
4,242434,1.0


In [77]:
patientIDs_IV = t_sus.patientunitstayid.unique().tolist()
patientIDs_IV_sub = pd.DataFrame(patientIDs_IV)
patientIDs_IV_sub.columns=['patientunitstayid']

Let's start the merging

In [78]:
labs_sus=pd.merge(lab_sofa,patientIDs_IV_sub,how="inner",on='patientunitstayid').drop_duplicates()
vent_sus=pd.merge(vent_sofa,patientIDs_IV_sub,how="inner",on='patientunitstayid').drop_duplicates()
gcs_sofa_sus=pd.merge(gcs_sofa,patientIDs_IV_sub,how="inner",on='patientunitstayid').drop_duplicates()
labs_vent=pd.merge(labs_sus,vent_sus,on=['patientunitstayid','offset'],how="outer").drop_duplicates()
final_sofa=pd.merge(labs_vent,gcs_sofa_sus,on=['patientunitstayid','offset'],how="outer").drop_duplicates()
final_sofa=final_sofa.fillna(0)
final_sofa['Total_SOFA']=final_sofa['SOFA_Coagulation']+final_sofa['SOFA_Liver']+final_sofa['SOFA_Respiration']+final_sofa['SOFA_Renal']+final_sofa['SOFA_cardio']+final_sofa['SOFA_GCS']
del lab_sofa
del vent_sofa
del gcs_sofa
del t_sus

In [79]:
result=final_sofa.groupby(['patientunitstayid'],as_index=False).apply(pd.DataFrame.sort_values,'offset')

In [80]:
final_sofa['diff_per_offset']=final_sofa.groupby(['patientunitstayid'])['Total_SOFA'].transform(lambda x: x.diff()).fillna(0)

In [84]:
final_sofa.groupby(['patientunitstayid']).sort_values(by=['offset'])

AttributeError: Cannot access callable attribute 'sort_values' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [85]:
#To do:
#Merge same offset results into a single row
#Sort the offset per pid
#Choose the rows where diff>=2
#Choose the row where offset<24 hours per pid
#compare with t_sus
#be happy


In [81]:
result.head()

patientunitstayid  offset  SOFA_Coagulation  SOFA_Liver  \
0 0                   242040    27.0               0.0         0.0   
  1370180             242040    69.0               0.0         0.0   
  1370181             242040    69.0               0.0         0.0   
  1370182             242040    69.0               0.0         0.0   
  1370183             242040   475.0               0.0         0.0   

           SOFA_Respiration  SOFA_Renal  SOFA_cardio  SOFA_GCS  Total_SOFA  
0 0                     0.0         0.0          0.0       0.0         0.0  
  1370180               0.0         0.0          0.0       4.0         4.0  
  1370181               0.0         0.0          0.0       0.0         0.0  
  1370182               0.0         0.0          0.0       3.0         3.0  
  1370183               0.0         0.0          0.0       4.0         4.0

In [74]:
result.columns=result.columns.droplevel(1)

AttributeError: 'Index' object has no attribute 'droplevel'

In [67]:
len(final_sofa.patientunitstayid.unique())

28166

In [86]:
sample=final_sofa.loc[final_sofa['patientunitstayid']==242040]

In [90]:
sample.groupby(['patientunitstayid','offset'])['SOFA_GCS'].apply(lambda x:x.sum()).reset_index()

,patientunitstayid,offset,SOFA_GCS
0,242040,27.0,0.0
1,242040,69.0,7.0
2,242040,475.0,7.0
3,242040,1061.0,0.0
4,242040,2567.0,0.0
5,242040,3952.0,0.0
6,242040,4757.0,0.0
7,242040,5374.0,0.0


In [82]:
final_sofa

,patientunitstayid,offset,SOFA_Coagulation,SOFA_Liver,SOFA_Respiration,SOFA_Renal,SOFA_cardio,SOFA_GCS,Total_SOFA,diff_per_offset
0,242040,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,242040,1061.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,242040,2567.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,242040,3952.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,242040,4757.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,242040,5374.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,242154,165.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0
7,242154,485.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0
8,242154,1030.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0
9,242154,1605.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0
